# Keras를 이용한 손글씨 분류하기

## MNIST 훈련데이터 준비하기

In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf

In [ ]:
mnist = tf.keras.datasets.mnist   
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
print(train_images.shape, train_labels.shape)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
## <샘플이미지 확인하기> 
## 넘파이 배열의 원소값이 0에 가까울수록 이미지가 검게 그려짐. 
## x_train_all[i] 의 i값을 다양하게 입력해 보세요. 0<=i<=59,999
plt.imshow(train_images[0], cmap='gray')
plt.show()

In [ ]:
## <Target 값 확인하기>
## y_target_all : 60,000개의 요소를 가진 1차원 배열. 각 요소는 0~9까지의 정수로 이루어져 있음. 
## 일단 가장 앞에 있는 10개만 출력함.
print(train_labels[:10])

In [ ]:
## 훈련세트의 목표값들이 고르게 분포되어 있는 지 확인함
## bincount() : 배열에 있는 정수값의 등장횟수를 카운트함.
np.bincount(train_labels)

In [ ]:
## 이미지 데이터는 픽셀마다 0~255 사이의 값을 가짐.
## [데이터표준화] 255로 나누어 0~1 사이로 맞추는 것이 유리함.
train_images = train_images / 255
test_images = test_images / 255

## 케라스를 소개합니다.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
# 2차원배열 (28*28픽셀)의 이미지 포맷을 28*28=784 픽셀의 1차원배열로 변환함
model.add(Flatten(input_shape=(28,28)))
## 은닉층을 정의함. 유닛개수 100개
model.add(Dense(100, activation='relu'))
## 출력층을 정의함. 유닛수는 10개
model.add(Dense(10, activation='softmax'))

In [ ]:
## 최적화알고리즘은 adam, 손실함수는 크로스엔트로피, metrics : 훈련과정기록으로 정확도를 남기기 위함.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
## 훈련세트에서 측정한 값들을 History 클래스 객체에 담아서 반환함.
history = model.fit(train_images, train_labels, epochs=20)

In [ ]:
## history 딕셔너리의 측정지표들
print(history.history.keys())

In [ ]:
## 손실 추이
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss'])
plt.show()

In [ ]:
## 정확도 추이
plt.plot(history.history['accuracy'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss'])
plt.show()

In [ ]:
## Verbosity mode.0 = silent, 1 = progress bar
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)

In [ ]:
print(f'테스트 손실도 : {test_loss}')

In [ ]:
print(f'테스트 정확도 : {test_accuracy}')

In [ ]:
# Step04. 테스트용 데이터를 대입했을 때, 맞추지 못한 항목에 대해 상세 보고한다.
# (1) 맞추지 못한 갯수
# (2) 맞추지 못한 항목 : 인덱스, 예측한 숫자값, 실제 숫자값

def report_model_miss(model, X, Y):
    
    hit = 0
    miss = 0
    nCnt = 0
    misses = []

    predictions = model.predict(X)

    for i in range(len(Y)):
      prediction = np.argmax(predictions[i])
      if prediction == Y[i]:
        hit += 1
      else:
        miss += 1
        miss_item = [i, prediction, Y[i]]
        misses.append(miss_item)
    
    score = hit/len(X)*100        
    print(f'Score is {score}')
       
    print(f'The number of miss: {miss}')
    for i in range(len(misses)):
      print(f'[{misses[i][0]+1}] (예측값){misses[i][1]} (실제값){misses[i][2]}')

      digit_image = X[misses[i][0]].reshape(28, 28)
      plt.imshow(digit_image, cmap=matplotlib.cm.binary, interpolation="nearest")
      #plt.imshow(digit_image)
      #plt.colorbar()
      plt.axis("off")
      plt.show()   


In [ ]:
# Step04. 테스트용 데이터를 대입했을 때, 맞추지 못한 항목에 대해 상세 보고한다.
# (1) 맞추지 못한 갯수
# (2) 맞추지 못한 항목 : 인덱스, 예측한 숫자값, 실제 숫자값
report_model_miss(model, test_images, test_labels)